# Import Python Packages

In [ ]:
import boto3
import os
import numpy as np
import uuid
import cv2 as cv
from PIL import Image
from matplotlib import pyplot as plt
from tensorflow.keras.models import model_from_json
import rasterio
import fiona
from osgeo import gdal, ogr, osr
import geopandas as gpd
import tarfile
from zipfile import ZipFile

# Set Variables For Inference

In [ ]:
output_bucket = "csu-team04-inference-output"
input_bucket = "csu-team04-inference-input"
model_bucket_name = "csu-team04-inference-model"
model_object_name = "model.tar.gz"

# Set Functions

In [ ]:
# Models come with 2 files:
# JSON files contain the main model data
# H5 files contain model weights
def load_model(json_file, weights_file):
    with open(json_file, 'r') as file:
        json_content = file.read()
        
    model = model_from_json(json_content)
    model.load_weights(weights_file)
    
    return model

In [ ]:
# Overlay Mask on Image
colour = [66, 255, 73]
colour = [i/255.0 for i in colour]

def place_mask_on_image(image, mask):
    np.place(image[:, :, :], mask[:, :, :] >= 0.5, colour)
    return image

# Make Prediction

In [ ]:
def predict(prediction):
    prediction = prediction[0][:, :, :]
    prediction = np.repeat(prediction, 3, 2)
    return prediction

# Load Model From S3

In [ ]:
print("Loading model...")
model_path = f"/tmp/{model_object_name}"

# Download model file from S3 into temp directory
s3 = boto3.client("s3")
s3.download_file(model_bucket_name, model_object_name, model_path)

# Extract model from tarball
model_file = tarfile.open(model_path)
model_file.extractall("/tmp/extracted_model")
print(f"Model files: {os.listdir('/tmp/extracted_model/')}")
      
# Load model for inference
model = load_model("/tmp/extracted_model/model.json", "/tmp/extracted_model/model.h5")
print("Model loaded successfully!")

# Perform Inference

In [ ]:
output_directory = "/home/ec2-user/SageMaker/output"
shapefile_directory = "/home/ec2-user/SageMaker/Shapefiles"
s3_resource = boto3.resource('s3')

for obj in s3_resource.Bucket(input_bucket).objects.all():
    img_object_name = (obj.key)
    img_file_path = f"/tmp/{uuid.uuid4()}-{img_object_name}"
    
    # Download image from S3 
    s3.download_file(input_bucket, img_object_name, img_file_path)
    original_img = Image.open(img_file_path)
    original_img_size = original_img.size
    image = cv.imread(img_file_path)
    
    # Format image for inference
    print("Formatting image...")
    img_shape = (512, 512)
    image = image[:,:,:3]
    image = np.expand_dims(cv.resize(image, img_shape), 0)
    image = (image/255.0)
    image = image.astype(np.float32)
    display(original_img)
    
    # Make Prediction
    prediction = predict(model(image))
    upload_path = f"{output_directory}/{img_object_name}"
    
    # Convert Results to Tiff
    result_image = Image.fromarray((prediction * 255).astype(np.uint8))
    result_image.save(upload_path)
    print(f"Saving tiff to {upload_path}")
    tiff_mask = Image.open(upload_path)
    display(tiff_mask)
    
    # Resize predicted mask image
    print("Resizing mask to original size")
    tiff_mask = tiff_mask.resize(original_img_size)
    tiff_mask.save(upload_path)
    
    # Copy geospatial data to predicted mask
    command = f"python /home/ec2-user/SageMaker/kernels/csu-team04-kernel/lib/python3.6/site-packages/osgeo_utils/samples/gdalcopyproj.py {img_file_path} {upload_path}"
    print(f"Running command {command}")
    os.system(command)

    # Display Overlay
    plt.subplot(1, 2, 1)
    plt.title("Satellite")
    plt.imshow(image[0])
    plt.subplot(1, 2, 2)
    plt.title("Predicted Mask")
    prediction = place_mask_on_image(image[0], prediction)
    plt.imshow(prediction)
    plt.show()
    
    # Convert tiff output mask to shapefile
    shapefile_name = img_object_name.split('.')[0]
    output_file = f"{shapefile_directory}/{shapefile_name}.shp"
    command = f"gdal_polygonize.py {upload_path} {output_file}"
    print(f"executing command {command}")
    os.system(command)
    
    # Send shapefile contents to zip
    print(f"Zipping contents of {shapefile_name}")
    with ZipFile(f"{shapefile_directory}/{shapefile_name}.zip", "w") as shapefile_zip:
        shapefile_zip.write(f"{shapefile_directory}/{shapefile_name}.shp")
        shapefile_zip.write(f"{shapefile_directory}/{shapefile_name}.dbf")
        shapefile_zip.write(f"{shapefile_directory}/{shapefile_name}.prj")
        shapefile_zip.write(f"{shapefile_directory}/{shapefile_name}.shx")
   
    # Upload Result to Bucket
    s3.upload_file(f"{shapefile_directory}/{shapefile_name}.zip", output_bucket, f"{shapefile_name}.zip")
    print("Uploaded shapefile zip to output bucket!")
    
    # Delete input image from bucket
    s3.delete_object(Bucket=input_bucket, Key=obj.key)
    print("Deleted image from input bucket!")
print("Inference complete!")

# Delete Output + Shapefile Contents

In [ ]:
!rm /home/ec2-user/SageMaker/output/* -rf
!rm /home/ec2-user/SageMaker/Shapefiles/* -rf